# Graph-Learning-Based Recommender System on MovieLens

### Group 9

- AGARWAL, Sahil
- WEI, Yuanjing
- ZHANG, Yujun yzhanglo@connect.ust.hk

Group project of COMP4222@HKUST in 2022 Fall.

# 1 Environment Configuration

In [1]:
# change the path in the following
try:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/MyDrive/4222Group9'
except:
    %cd '/data/yzhanglo/4222project'

import comp4222
import recommenders
%pwd
%ls

/data/yzhanglo/4222project
comp4222/     lightgcn.yaml     movielens.ipynb  requirements.txt
cuda101.yml   logs/             movielens.zip    sahil_main.ipynb
KGAT_folder/  main.ipynb        params.csv       save1.html
LICENSE       ml-100k/          README.md        tp.ipynb
LightGCN/     ml-latest-small/  recommenders/    zory_main.ipynb


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
from tensorboardX import SummaryWriter

# easier to print by putting variable as a single line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# make matplotlib figures appear inline in the notebook rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.2f}'.format
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]

def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols

# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

2022-12-13 23:31:46.823967: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/data/yzhanglo/miniconda/envs/cu101/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Testing the module functionality
from comp4222 import b
comp4222.b.ok()

hahaha


# 2 MovieLens


We're using ml-100k from MovieLens. It contains 100000 ratings from 943 users on 1682 movies. Each user has rated at least 20 movies. And the data was collected during the seven-month period from September 19th, 1997 through April 22nd, 1998. The readme.md is avaliable [here](https://files.grouplens.org/datasets/movielens/ml-100k-README.txt).

P.s. As for now, we're using the provided code for dataset loading defined by lightgcn and microsoft recommender repository.

## Data Loading

In [ ]:
# Download MovieLens data.
dataset_name = "ml-latest-small"
from urllib.request import urlretrieve
import zipfile
urlretrieve(f"https://files.grouplens.org/datasets/movielens/{dataset_name}.zip", "movielens.zip")
zipfile.ZipFile("movielens.zip", "r").extractall()

In [ ]:
movies = pd.read_csv(f"{dataset_name}/movies.csv")
genre_cols = [
    "(no genres listed)", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies

In [ ]:
tags = pd.read_csv(f"{dataset_name}/tags.csv")
tags

In [ ]:
ratings = pd.read_csv(f"{dataset_name}/ratings.csv")
ratings

## Data Exploration

In [ ]:
#%pip install altair
import altair as alt
alt.data_transformers.enable('default', max_rows=None)
alt.renderers.enable('colab')

# 3 Preliminaries

There's no much symbol to be defined at this stage.

# 4 Models Definition

In [4]:
from recommenders.utils.timer import Timer
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
from recommenders.utils.constants import (
COL_DICT,
    DEFAULT_K,
    DEFAULT_USER_COL,
    DEFAULT_ITEM_COL,
    DEFAULT_RATING_COL,
    DEFAULT_PREDICTION_COL,
    DEFAULT_TIMESTAMP_COL,
    SEED,
)

# Helpers
import os
from tempfile import TemporaryDirectory
tmp_dir = TemporaryDirectory()
TRAIN_FILE = os.path.join(tmp_dir.name, "df_train.csv")
TEST_FILE = os.path.join(tmp_dir.name, "df_test.csv")

from recommenders.evaluation.python_evaluation import (
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
)
def ranking_metrics_python(test, predictions, k=DEFAULT_K):
    return {
        "MAP": map_at_k(test, predictions, k=k, **COL_DICT),
        "nDCG@k": ndcg_at_k(test, predictions, k=k, **COL_DICT),
        "Precision@k": precision_at_k(test, predictions, k=k, **COL_DICT),
        "Recall@k": recall_at_k(test, predictions, k=k, **COL_DICT),
    }


In [5]:
from recommenders.utils.spark_utils import start_or_get_spark
spark = start_or_get_spark("PySpark", memory="32g")
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")

from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
import os

# fix random seeds to make sure out runs are reproducible
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/13 23:32:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## ALS

In [14]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, LongType
from pyspark.ml.recommendation import ALS


def prepare_training_als(train, test):
    schema = StructType(
        (
            StructField(DEFAULT_USER_COL, IntegerType()),
            StructField(DEFAULT_ITEM_COL, IntegerType()),
            StructField(DEFAULT_RATING_COL, FloatType()),
            StructField(DEFAULT_TIMESTAMP_COL, LongType()),
        )
    )
    spark = start_or_get_spark()
    return spark.createDataFrame(train, schema).cache()

def prepare_metrics_als(train, test):
    schema = StructType(
        (
            StructField(DEFAULT_USER_COL, IntegerType()),
            StructField(DEFAULT_ITEM_COL, IntegerType()),
            StructField(DEFAULT_RATING_COL, FloatType()),
            StructField(DEFAULT_TIMESTAMP_COL, LongType()),
        )
    )
    spark = start_or_get_spark()
    return spark.createDataFrame(train, schema).cache(), spark.createDataFrame(test, schema).cache()

def predict_als(model, test):
    with Timer() as t:
        preds = model.transform(test)
    return preds, t

def train_als(params, data):
    symbol = ALS(**params)
    with Timer() as t:
        model = symbol.fit(data)
    return model, t

def recommend_k_als(model, test, train, top_k=DEFAULT_K, remove_seen=True):
    with Timer() as t:
        # Get the cross join of all user-item pairs and score them.
        users = train.select(DEFAULT_USER_COL).distinct()
        items = train.select(DEFAULT_ITEM_COL).distinct()
        user_item = users.crossJoin(items)
        dfs_pred = model.transform(user_item)

        # Remove seen items
        dfs_pred_exclude_train = dfs_pred.alias("pred").join(
            train.alias("train"),
            (dfs_pred[DEFAULT_USER_COL] == train[DEFAULT_USER_COL])
            & (dfs_pred[DEFAULT_ITEM_COL] == train[DEFAULT_ITEM_COL]),
            how="outer",
        )
        topk_scores = dfs_pred_exclude_train.filter(
            dfs_pred_exclude_train["train." + DEFAULT_RATING_COL].isNull()
        ).select(
            "pred." + DEFAULT_USER_COL,
            "pred." + DEFAULT_ITEM_COL,
            "pred." + DEFAULT_PREDICTION_COL,
        )
    return topk_scores, t


als_params = {
    "rank": 10,
    "maxIter": 20,
    "implicitPrefs": False,
    "alpha": 0.1,
    "regParam": 0.05,
    "coldStartStrategy": "drop",
    "nonnegative": False,
    "userCol": DEFAULT_USER_COL,
    "itemCol": DEFAULT_ITEM_COL,
    "ratingCol": DEFAULT_RATING_COL,
}

## NCF

In [15]:
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset

def prepare_training_ncf(df_train, df_test):
    #df_train.sort_values(["userID"], axis=0, ascending=[True], inplace=True)
    #df_test.sort_values(["userID"], axis=0, ascending=[True], inplace=True)
    train = df_train.sort_values(["userID"], axis=0, ascending=[True])
    test = df_test.sort_values(["userID"], axis=0, ascending=[True])
    test = test[df_test["userID"].isin(train["userID"].unique())]
    test = test[test["itemID"].isin(train["itemID"].unique())]
    train.to_csv(TRAIN_FILE, index=False)
    test.to_csv(TEST_FILE, index=False)
    return NCFDataset(
        train_file=TRAIN_FILE,
        col_user=DEFAULT_USER_COL,
        col_item=DEFAULT_ITEM_COL,
        col_rating=DEFAULT_RATING_COL,
        seed=SEED,
    )


def train_ncf(params, data):
    model = NCF(n_users=data.n_users, n_items=data.n_items, **params)
    with Timer() as t:
        model.fit(data)
    return model, t


def recommend_k_ncf(model, test, train, top_k=DEFAULT_K, remove_seen=True):
    with Timer() as t:
        users, items, preds = [], [], []
        item = list(train[DEFAULT_ITEM_COL].unique())
        for user in train[DEFAULT_USER_COL].unique():
            user = [user] * len(item)
            users.extend(user)
            items.extend(item)
            preds.extend(list(model.predict(user, item, is_list=True)))
        topk_scores = pd.DataFrame(
            data={
                DEFAULT_USER_COL: users,
                DEFAULT_ITEM_COL: items,
                DEFAULT_PREDICTION_COL: preds,
            }
        )
        merged = pd.merge(
            train, topk_scores, on=[DEFAULT_USER_COL, DEFAULT_ITEM_COL], how="outer"
        )
        topk_scores = merged[merged[DEFAULT_RATING_COL].isnull()].drop(
            DEFAULT_RATING_COL, axis=1
        )
    # Remove temp files
    return topk_scores, t

ncf_params = {
    "model_type": "NeuMF",
    "n_factors": 4,
    "layer_sizes": [16, 8, 4],
    "n_epochs": 20,
    "batch_size": 1024,
    "learning_rate": 1e-3,
    "verbose": 10
}

## KGAT

In [ ]:
#%pip install easydict
import os
import sys
import random
from time import time

import pandas as pd
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim

from KGAT_folder.KGAT import KGAT
from KGAT_folder.log_helper import *
from KGAT_folder.parser_kgat import *
from KGAT_folder.metrics import *
from KGAT_folder.model_helper import *
from KGAT_folder.loader_kgat import DataLoaderKGAT


def train_kgat():
    args = parse_kgat_args()
    #train(args)
    time0 = time()
    model, data, Ks, device = train(args)
    time1 = time()
    t = time1-time0
    return model, data, Ks, device, t

def evaluate_kgat(model, dataloader, Ks, device):
    test_batch_size = dataloader.test_batch_size
    train_user_dict = dataloader.train_user_dict
    test_user_dict = dataloader.test_user_dict

    model.eval()

    user_ids = list(test_user_dict.keys())
    user_ids_batches = [user_ids[i: i + test_batch_size] for i in range(0, len(user_ids), test_batch_size)]
    user_ids_batches = [torch.LongTensor(d) for d in user_ids_batches]

    n_items = dataloader.n_items
    item_ids = torch.arange(n_items, dtype=torch.long).to(device)

    cf_scores = []
    metric_names = ['precision', 'recall', 'ndcg']
    metrics_dict = {k: {m: [] for m in metric_names} for k in Ks}

    with tqdm(total=len(user_ids_batches), desc='Evaluating Iteration') as pbar:
        for batch_user_ids in user_ids_batches:
            batch_user_ids = batch_user_ids.to(device)

            with torch.no_grad():
                batch_scores = model(batch_user_ids, item_ids, mode='predict')       # (n_batch_users, n_items)

            batch_scores = batch_scores.cpu()
            batch_metrics = calc_metrics_at_k(batch_scores, train_user_dict, test_user_dict, batch_user_ids.cpu().numpy(), item_ids.cpu().numpy(), Ks)

            cf_scores.append(batch_scores.numpy())
            for k in Ks:
                for m in metric_names:
                    metrics_dict[k][m].append(batch_metrics[k][m])
            pbar.update(1)

    cf_scores = np.concatenate(cf_scores, axis=0)
    for k in Ks:
        for m in metric_names:
            metrics_dict[k][m] = np.concatenate(metrics_dict[k][m]).mean()
    return cf_scores, metrics_dict


def train(args):
    # seed
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

    #log_save_id = create_log_id(args.save_dir)
    #logging_config(folder=args.save_dir, name='log{:d}'.format(log_save_id), no_console=False)
    #logging.info(args)

    # GPU / CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # load data
    data = DataLoaderKGAT(args, logging)
    if args.use_pretrain == 1:
        user_pre_embed = torch.tensor(data.user_pre_embed)
        item_pre_embed = torch.tensor(data.item_pre_embed)
    else:
        user_pre_embed, item_pre_embed = None, None

    # construct model & optimizer
    model = KGAT(args, data.n_users, data.n_entities, data.n_relations, data.A_in, user_pre_embed, item_pre_embed)
    #if args.use_pretrain == 2:
    #    model = load_model(model, args.pretrain_model_path)

    model.to(device)
    #logging.info(model)

    cf_optimizer = optim.Adam(model.parameters(), lr=args.lr)
    kg_optimizer = optim.Adam(model.parameters(), lr=args.lr)

    # initialize metrics
    best_epoch = -1
    best_recall = 0

    Ks = eval(args.Ks)
    k_min = min(Ks)
    k_max = max(Ks)

    epoch_list = []
    metrics_list = {k: {'precision': [], 'recall': [], 'ndcg': []} for k in Ks}

    # train model
    for epoch in range(1, args.n_epoch + 1):
        time0 = time()
        model.train()

        # train cf
        time1 = time()
        cf_total_loss = 0
        n_cf_batch = data.n_cf_train // data.cf_batch_size + 1

        for iter in range(1, n_cf_batch + 1):
            time2 = time()
            cf_batch_user, cf_batch_pos_item, cf_batch_neg_item = data.generate_cf_batch(data.train_user_dict, data.cf_batch_size)
            cf_batch_user = cf_batch_user.to(device)
            cf_batch_pos_item = cf_batch_pos_item.to(device)
            cf_batch_neg_item = cf_batch_neg_item.to(device)

            cf_batch_loss = model(cf_batch_user, cf_batch_pos_item, cf_batch_neg_item, mode='train_cf')

            if np.isnan(cf_batch_loss.cpu().detach().numpy()):
                logging.info('ERROR (CF Training): Epoch {:04d} Iter {:04d} / {:04d} Loss is nan.'.format(epoch, iter, n_cf_batch))
                sys.exit()

            cf_batch_loss.backward()
            cf_optimizer.step()
            cf_optimizer.zero_grad()
            cf_total_loss += cf_batch_loss.item()

            if (iter % args.cf_print_every) == 0:
                logging.info('CF Training: Epoch {:04d} Iter {:04d} / {:04d} | Time {:.1f}s | Iter Loss {:.4f} | Iter Mean Loss {:.4f}'.format(epoch, iter, n_cf_batch, time() - time2, cf_batch_loss.item(), cf_total_loss / iter))
        logging.info('CF Training: Epoch {:04d} Total Iter {:04d} | Total Time {:.1f}s | Iter Mean Loss {:.4f}'.format(epoch, n_cf_batch, time() - time1, cf_total_loss / n_cf_batch))

        # train kg
        time3 = time()
        kg_total_loss = 0
        n_kg_batch = data.n_kg_train // data.kg_batch_size + 1

        for iter in range(1, n_kg_batch + 1):
            time4 = time()
            kg_batch_head, kg_batch_relation, kg_batch_pos_tail, kg_batch_neg_tail = data.generate_kg_batch(data.train_kg_dict, data.kg_batch_size, data.n_users_entities)
            kg_batch_head = kg_batch_head.to(device)
            kg_batch_relation = kg_batch_relation.to(device)
            kg_batch_pos_tail = kg_batch_pos_tail.to(device)
            kg_batch_neg_tail = kg_batch_neg_tail.to(device)

            kg_batch_loss = model(kg_batch_head, kg_batch_relation, kg_batch_pos_tail, kg_batch_neg_tail, mode='train_kg')

            if np.isnan(kg_batch_loss.cpu().detach().numpy()):
                logging.info('ERROR (KG Training): Epoch {:04d} Iter {:04d} / {:04d} Loss is nan.'.format(epoch, iter, n_kg_batch))
                sys.exit()

            kg_batch_loss.backward()
            kg_optimizer.step()
            kg_optimizer.zero_grad()
            kg_total_loss += kg_batch_loss.item()

            if (iter % args.kg_print_every) == 0:
                logging.info('KG Training: Epoch {:04d} Iter {:04d} / {:04d} | Time {:.1f}s | Iter Loss {:.4f} | Iter Mean Loss {:.4f}'.format(epoch, iter, n_kg_batch, time() - time4, kg_batch_loss.item(), kg_total_loss / iter))
        logging.info('KG Training: Epoch {:04d} Total Iter {:04d} | Total Time {:.1f}s | Iter Mean Loss {:.4f}'.format(epoch, n_kg_batch, time() - time3, kg_total_loss / n_kg_batch))

        # update attention
        time5 = time()
        h_list = data.h_list.to(device)
        t_list = data.t_list.to(device)
        r_list = data.r_list.to(device)
        relations = list(data.laplacian_dict.keys())
        model(h_list, t_list, r_list, relations, mode='update_att')
        logging.info('Update Attention: Epoch {:04d} | Total Time {:.1f}s'.format(epoch, time() - time5))

        logging.info('CF + KG Training: Epoch {:04d} | Total Time {:.1f}s'.format(epoch, time() - time0))
    return model, data, Ks, device


In [ ]:
print(f"\nComputing {'KGAT'} algorithm on Movielens {'100k'}")
model, data, Ks, device, time_train = train_kgat()
print('time_train: ', time_train)
_, metrics_dict_kgat = evaluate_kgat(model, data, Ks, device)
print(metrics_dict_kgat)

## LightGCN

### Tuning alpha_ks(old)

In [16]:
from LightGCN.code.main_lgcn import run_lightgcn
# input 0: original layer-stacking weights, 1: modified layer-stacking weights
run_lightgcn(0)

INFO:py4j.clientserver:Error while sending or receiving.
Traceback (most recent call last):
  File "/data/yzhanglo/miniconda/envs/cu101/lib/python3.8/site-packages/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer
INFO:py4j.clientserver:Closing down clientserver connection
INFO:root:Exception while sending command.
Traceback (most recent call last):
  File "/data/yzhanglo/miniconda/envs/cu101/lib/python3.8/site-packages/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/yzhanglo/miniconda/envs/cu101/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/data/yzhanglo/miniconda/envs

stacking_func: 0
use NORMAL distribution initilizer
loading adjacency matrix
lgn is already to go(dropout:0)
load and save to ./LightGCN/code/checkpoints/lgn-movielens-3-64.pth.tar
we have tensorboard!
[TEST]


INFO:tensorboardX.summary:Summary name Test/Precision@[20, 40, 60] is illegal; using Test/Precision__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Precision@[20, 40, 60] is illegal; using Test/Precision__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/NDCG@[20, 40, 60] is illegal; using Test/NDCG__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/NDCG@[20, 40, 60] is illegal; using Test/NDCG__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/NDCG@[20, 40, 60] is illegal; using Test/NDCG__20__40__60_ instead.


{'precision': array([0.01765351, 0.01781798, 0.01754386]), 'recall': array([0.0146577 , 0.0286584 , 0.04074619]), 'ndcg': array([0.02030313, 0.02466048, 0.02908808])}
EPOCH[1/500] loss0.690-|Sample:1.04|
EPOCH[2/500] loss0.675-|Sample:0.66|
EPOCH[3/500] loss0.609-|Sample:0.49|
EPOCH[4/500] loss0.501-|Sample:0.72|
EPOCH[5/500] loss0.413-|Sample:0.49|
EPOCH[6/500] loss0.365-|Sample:0.49|
EPOCH[7/500] loss0.330-|Sample:0.73|
EPOCH[8/500] loss0.315-|Sample:0.49|
EPOCH[9/500] loss0.303-|Sample:0.57|


INFO:tensorboardX.summary:Summary name Test/Recall@[20, 40, 60] is illegal; using Test/Recall__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Recall@[20, 40, 60] is illegal; using Test/Recall__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Recall@[20, 40, 60] is illegal; using Test/Recall__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Precision@[20, 40, 60] is illegal; using Test/Precision__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Precision@[20, 40, 60] is illegal; using Test/Precision__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Precision@[20, 40, 60] is illegal; using Test/Precision__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/NDCG@[20, 40, 60] is illegal; using Test/NDCG__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/NDCG@[20, 40, 60] is illegal; using Test/NDCG__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/NDCG@[20, 40, 60] 

...

EPOCH[470/500] loss0.090-|Sample:0.79|
[TEST]
{'precision': array([0.28596491, 0.221875  , 0.18600146]), 'recall': array([0.32006744, 0.46331328, 0.55692915]), 'ndcg': array([0.40024274, 0.42099204, 0.44494205])}
EPOCH[471/500] loss0.085-|Sample:0.50|
EPOCH[472/500] loss0.088-|Sample:0.49|
EPOCH[473/500] loss0.090-|Sample:0.71|
EPOCH[474/500] loss0.087-|Sample:0.50|
EPOCH[475/500] loss0.086-|Sample:0.49|
EPOCH[476/500] loss0.088-|Sample:0.72|
EPOCH[477/500] loss0.089-|Sample:0.57|
EPOCH[478/500] loss0.088-|Sample:0.82|
EPOCH[479/500] loss0.088-|Sample:0.49|


INFO:tensorboardX.summary:Summary name Test/Recall@[20, 40, 60] is illegal; using Test/Recall__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Recall@[20, 40, 60] is illegal; using Test/Recall__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Recall@[20, 40, 60] is illegal; using Test/Recall__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Precision@[20, 40, 60] is illegal; using Test/Precision__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Precision@[20, 40, 60] is illegal; using Test/Precision__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Precision@[20, 40, 60] is illegal; using Test/Precision__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/NDCG@[20, 40, 60] is illegal; using Test/NDCG__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/NDCG@[20, 40, 60] is illegal; using Test/NDCG__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/NDCG@[20, 40, 60] 

EPOCH[480/500] loss0.088-|Sample:0.49|
[TEST]
{'precision': array([0.28629386, 0.22247807, 0.18585526]), 'recall': array([0.3206053 , 0.46355916, 0.55493971]), 'ndcg': array([0.40017314, 0.421211  , 0.44420483])}
EPOCH[481/500] loss0.089-|Sample:0.73|
EPOCH[482/500] loss0.089-|Sample:0.57|
EPOCH[483/500] loss0.089-|Sample:0.71|
EPOCH[484/500] loss0.086-|Sample:0.50|
EPOCH[485/500] loss0.089-|Sample:0.79|
EPOCH[486/500] loss0.090-|Sample:0.67|
EPOCH[487/500] loss0.088-|Sample:0.49|
EPOCH[488/500] loss0.089-|Sample:0.93|
EPOCH[489/500] loss0.085-|Sample:0.66|


INFO:tensorboardX.summary:Summary name Test/Recall@[20, 40, 60] is illegal; using Test/Recall__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Recall@[20, 40, 60] is illegal; using Test/Recall__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Recall@[20, 40, 60] is illegal; using Test/Recall__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Precision@[20, 40, 60] is illegal; using Test/Precision__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Precision@[20, 40, 60] is illegal; using Test/Precision__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/Precision@[20, 40, 60] is illegal; using Test/Precision__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/NDCG@[20, 40, 60] is illegal; using Test/NDCG__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/NDCG@[20, 40, 60] is illegal; using Test/NDCG__20__40__60_ instead.
INFO:tensorboardX.summary:Summary name Test/NDCG@[20, 40, 60] 

EPOCH[490/500] loss0.087-|Sample:0.84|
[TEST]
{'precision': array([0.28563596, 0.22220395, 0.18618421]), 'recall': array([0.31907928, 0.46283085, 0.55576431]), 'ndcg': array([0.40027067, 0.42156901, 0.44530608])}
EPOCH[491/500] loss0.085-|Sample:0.51|
EPOCH[492/500] loss0.088-|Sample:0.49|
EPOCH[493/500] loss0.083-|Sample:0.62|
EPOCH[494/500] loss0.087-|Sample:0.72|
EPOCH[495/500] loss0.088-|Sample:0.49|
EPOCH[496/500] loss0.087-|Sample:0.51|
EPOCH[497/500] loss0.085-|Sample:0.77|
EPOCH[498/500] loss0.086-|Sample:0.49|
EPOCH[499/500] loss0.084-|Sample:0.60|
EPOCH[500/500] loss0.088-|Sample:0.89|


In [ ]:
#from LightGCN.code.main_lgcn import run_lightgcn
#run_lightgcn(1)

In [ ]:
from LightGCN.code.main_lgcn import run_lightgcn
run_lightgcn(1.5)

In [ ]:
#from LightGCN.code.main_lgcn import run_lightgcn
#run_lightgcn(2)

In [ ]:
from LightGCN.code.main_lgcn import run_lightgcn
run_lightgcn(3)

Cpp extension not loaded
>>SEED: 2020
loading [LightGCN/data/movielens]
44140 interactions for training
11235 interactions for testing
movielens Sparsity : 0.03502087022514546
movielens is ready to go
===========config================
{'A_n_fold': 100,
 'A_split': False,
 'bigdata': False,
 'bpr_batch_size': 64,
 'decay': 0.0001,
 'dropout': 0,
 'keep_prob': 0.6,
 'latent_dim_rec': 64,
 'lightGCN_n_layers': 3,
 'lr': 0.0001,
 'multicore': 0,
 'pretrain': 0,
 'stacking_func': 1,
 'test_u_batch_size': 45}
cores for test: 10
comment: lgn
tensorboard: 1
LOAD: 0
Weight path: ./checkpoints
Test Topks: [20, 40, 60]
using bpr loss
===========end===================
stacking_func: 3
use NORMAL distribution initilizer
loading adjacency matrix
successfully loaded...


### For comparison

In [ ]:
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN

def prepare_training_lightgcn(train, test):
    return ImplicitCF(train=train, test=test)

def train_lightgcn(params, data):
    hparams = prepare_hparams(**params)
    model = LightGCN(hparams, data)
    with Timer() as t:
        model.fit()
    return model, t

def recommend_k_lightgcn(model, test, train, top_k=DEFAULT_K, remove_seen=True):
    with Timer() as t:
        topk_scores = model.recommend_k_items(
            test, top_k=top_k, remove_seen=remove_seen
        )
    return topk_scores, t

lightgcn_param = {
    "yaml_file": os.path.join("drive", "MyDrive", "4222Group9", "lightgcn.yaml"),
    "n_layers": 3,
    "batch_size": 1024,
    "epochs": 10,
    "learning_rate": 0.005,
    "eval_epoch": 10,
    "top_k": DEFAULT_K,
}

## Comparison

In [ ]:
params = {
    "als": als_params,
    "ncf": ncf_params,
    "lightgcn": lightgcn_param,
}
prepare_training_data = {
    "als": prepare_training_als,
    "ncf": prepare_training_ncf,
    "lightgcn": prepare_training_lightgcn,
}


from recommenders.evaluation.spark_evaluation import (
    SparkRatingEvaluation,
    SparkRankingEvaluation,
)
def rating_metrics_pyspark(test, predictions):
    rating_eval = SparkRatingEvaluation(test, predictions, **COL_DICT)
    return {
        "RMSE": rating_eval.rmse(),
        "MAE": rating_eval.mae(),
        "R2": rating_eval.exp_var(),
        "Explained Variance": rating_eval.rsquared(),
    }
def ranking_metrics_pyspark(test, predictions, k=DEFAULT_K):
    rank_eval = SparkRankingEvaluation(
        test, predictions, k=k, relevancy_method="top_k", **COL_DICT
    )
    return {
        "MAP": rank_eval.map_at_k(),
        "nDCG@k": rank_eval.ndcg_at_k(),
        "Precision@k": rank_eval.precision_at_k(),
        "Recall@k": rank_eval.recall_at_k(),
    }

prepare_metrics_data = {
    "als": lambda train, test: prepare_metrics_als(train, test),
}
trainer = {
    "als": lambda params, data: train_als(params, data),
    "ncf": lambda params, data: train_ncf(params, data),
    "lightgcn": lambda params, data: train_lightgcn(params, data),
}
rating_predictor = {
    "als": lambda model, test: predict_als(model, test),
}
rating_evaluator = {
    "als": lambda test, predictions: rating_metrics_pyspark(test, predictions)
}
ranking_predictor = {
    "als": lambda model, test, train: recommend_k_als(model, test, train),
    "ncf": lambda model, test, train: recommend_k_ncf(model, test, train),
    "lightgcn": lambda model, test, train: recommend_k_lightgcn(model, test, train),
}
ranking_evaluator = {
    "als": lambda test, predictions, k: ranking_metrics_pyspark(test, predictions, k),
    "ncf": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
    "lightgcn": lambda test, predictions, k: ranking_metrics_python(test, predictions, k),
}
metrics = {
    "als": ["rating", "ranking"],
    "ncf": ["ranking"],
    "lightgcn": ["ranking"]
}

# 5 Hyperparameter Tunning

In [ ]:
def generate_summary(data, algo, k, train_time, time_rating, rating_metrics, time_ranking, ranking_metrics):
    summary = {"Data": data, "Algo": algo, "K": k, "Train time (s)": train_time, "Predicting time (s)": time_rating, "Recommending time (s)": time_ranking}
    if rating_metrics is None:
        rating_metrics = {
            "RMSE": np.nan,
            "MAE": np.nan,
            "R2": np.nan,
            "Explained Variance": np.nan,
        }
    if ranking_metrics is None:
        ranking_metrics = {
            "MAP": np.nan,
            "nDCG@k": np.nan,
            "Precision@k": np.nan,
            "Recall@k": np.nan,
        }
    summary.update(rating_metrics)
    summary.update(ranking_metrics)
    return summary

In [ ]:
data_sizes = ["100k"] # Movielens data size: 100k, 1m, 10m, or 20m
algorithms = [ "lightgcn"]

## Sanity Check by Overfitting on Small Data (skip)

In [ ]:
#!pip install pytorch-lightning
from pytorch_lightning import Trainer, seed_everything
from LightGCN.code.main_lgcn import sanity_check

seed_everything(42, workers=True)

model = sanity_check()
trainer = Trainer(max_epochs=10000, overfit_batches=0.01)
trainer.fit(model)

## Training

In [ ]:
%%time

data_sizes = ["100k"] # Movielens data size: 100k, 1m, 10m, or 20m
algorithms = [ "lightgcn"]

# For each data size and each algorithm, a recommender is evaluated. 
cols = ["Data", "Algo", "K", "Train time (s)", "Predicting time (s)", "RMSE", "MAE", "R2", "Explained Variance", "Recommending time (s)", "MAP", "nDCG@k", "Precision@k", "Recall@k"]
df_results = pd.DataFrame(columns=cols)

for data_size in data_sizes:
    # Load the dataset
    df = movielens.load_pandas_df(
        size=data_size,
        header=[DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL]
    )
    print("Size of Movielens {}: {}".format(data_size, df.shape))
    
    # Split the dataset
    df_train, df_test = python_stratified_split(df,
                                                ratio=0.75, 
                                                min_rating=1, 
                                                filter_by="item", 
                                                col_user=DEFAULT_USER_COL, 
                                                col_item=DEFAULT_ITEM_COL
                                                )
   
    # Loop through the algos
    for algo in algorithms:
        print(f"\nComputing {algo} algorithm on Movielens {data_size}")
        if algo == 'kgat':
            model, data, Ks, device, time_train = train_kgat()
            _, metrics_dict_kgat = evaluate_kgat(model, data, Ks, device)
            print(metrics_dict_kgat)
            # Record results
            #summary = generate_summary('100k', algo, DEFAULT_K, time_train, time_rating, ratings, time_ranking, rankings)
            #df_results.loc[df_results.shape[0] + 1] = summary
            
        else:
            # Data prep for training set
            train = prepare_training_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
            
            # Get model parameters
            model_params = params[algo]
            
            # Train the model
            model, time_train = trainer[algo](model_params, train)
            print(f"Training time: {time_train}s")
                    
            # Predict and evaluate
            train, test = prepare_metrics_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
            
            if "rating" in metrics[algo]:   
                # Predict for rating
                preds, time_rating = rating_predictor[algo](model, test)
                print(f"Rating prediction time: {time_rating}s")
                
                # Evaluate for rating
                ratings = rating_evaluator[algo](test, preds)
            else:
                ratings = None
                time_rating = np.nan
            
            if "ranking" in metrics[algo]:
                # Predict for ranking
                top_k_scores, time_ranking = ranking_predictor[algo](model, test, train)
                print(f"Ranking prediction time: {time_ranking}s")
                
                # Evaluate for rating
                rankings = ranking_evaluator[algo](test, top_k_scores, DEFAULT_K)
            else:
                rankings = None
                time_ranking = np.nan
                
            # Record results
            summary = generate_summary(data_size, algo, DEFAULT_K, time_train, time_rating, ratings, time_ranking, rankings)
            df_results.loc[df_results.shape[0] + 1] = summary
        
print("\nComputation finished")

## Training Plot

click "launch TensorBoard Session" in main_lgcn.py

# 6 Comparisons on Movielens and Movie

In [ ]:
data_sizes = ["100k","1m"] # Movielens data size: 100k, 1m, 10m, or 20m
#algorithms = [ "lightgcn"]
algorithms = ["als", "ncf", "lightgcn", "kgat"]

In [ ]:
%%time

# For each data size and each algorithm, a recommender is evaluated. 
cols = ["Data", "Algo", "K", "Train time (s)", "Predicting time (s)", "RMSE", "MAE", "R2", "Explained Variance", "Recommending time (s)", "MAP", "nDCG@k", "Precision@k", "Recall@k"]
df_results = pd.DataFrame(columns=cols)

for data_size in data_sizes:
    # Load the dataset
    df = movielens.load_pandas_df(
        size=data_size,
        header=[DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL]
    )
    print("Size of Movielens {}: {}".format(data_size, df.shape))
    
    # Split the dataset
    df_train, df_test = python_stratified_split(df,
                                                ratio=0.75, 
                                                min_rating=1, 
                                                filter_by="item", 
                                                col_user=DEFAULT_USER_COL, 
                                                col_item=DEFAULT_ITEM_COL
                                                )
   
    # Loop through the algos
    for algo in algorithms:
        print(f"\nComputing {algo} algorithm on Movielens {data_size}")
        if algo == 'kgat':
            model, data, Ks, device, time_train = train_kgat()
            _, metrics_dict_kgat = evaluate_kgat(model, data, Ks, device)
            print(metrics_dict_kgat)
            # Record results
            #summary = generate_summary('100k', algo, DEFAULT_K, time_train, time_rating, ratings, time_ranking, rankings)
            #df_results.loc[df_results.shape[0] + 1] = summary
            
        else:
            # Data prep for training set
            train = prepare_training_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
            
            # Get model parameters
            model_params = params[algo]
            
            # Train the model
            model, time_train = trainer[algo](model_params, train)
            print(f"Training time: {time_train}s")
                    
            # Predict and evaluate
            train, test = prepare_metrics_data.get(algo, lambda x,y:(x,y))(df_train, df_test)
            
            if "rating" in metrics[algo]:   
                # Predict for rating
                preds, time_rating = rating_predictor[algo](model, test)
                print(f"Rating prediction time: {time_rating}s")
                
                # Evaluate for rating
                ratings = rating_evaluator[algo](test, preds)
            else:
                ratings = None
                time_rating = np.nan
            
            if "ranking" in metrics[algo]:
                # Predict for ranking
                top_k_scores, time_ranking = ranking_predictor[algo](model, test, train)
                print(f"Ranking prediction time: {time_ranking}s")
                
                # Evaluate for rating
                rankings = ranking_evaluator[algo](test, top_k_scores, DEFAULT_K)
            else:
                rankings = None
                time_ranking = np.nan
                
            # Record results
            summary = generate_summary(data_size, algo, DEFAULT_K, time_train, time_rating, ratings, time_ranking, rankings)
            df_results.loc[df_results.shape[0] + 1] = summary
        
print("\nComputation finished")

## Print the result summary

In [ ]:
df_results

# 7 Credit and Reference

1. https://github.com/microsoft/recommenders